#Data Transition Cloud Server Side

##This code is used together with dtata transition Google coral side when we use some cloud computations, this functions must be running paralelly at the same time

In [ ]:
pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 19.9 MB/s eta 0:00:00


In [ ]:
import easyocr
import cv2
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import numpy

# Server Side

In [ ]:
import psycopg2
import pickle
import time
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import easyocr
import cv2
from matplotlib import pyplot as plt
import numpy as np

reader = easyocr.Reader(['ru'], gpu=True)

def poll_for_changes():
    # Open a connection
    conn = psycopg2.connect("host=109.120.191.148 dbname=PostgreSQL-4323 user=user password=0HD9E7L5A0410WYD")
    cursor = conn.cursor()


    while True:
        # Check the arrays table for a new array.
        cursor.execute("""
            SELECT id, image FROM arrays
            ORDER BY id DESC
            LIMIT 1
        """)
        row = cursor.fetchone()

        if row is not None:
            id, pickled_array = row
            array = pickle.loads(pickled_array)
            result = reader.readtext(array)
            finalres=result[0][1].lower()+str(result[2][1])

            finalres = list(finalres)
            for i in range (1,4):
              if finalres[i]=='о':
                finalres[i]=str(0)
            car_plate = ''.join(finalres)
            
            # The specific car plate number we're looking for

            # Check the users table for a user with the specified car plate number
            cursor.execute("""
                SELECT user_id FROM users WHERE car_plate = %s
            """, (car_plate,))
            user = cursor.fetchone()

            # If the user exists, add a new row to the results table
            if user is not None:
                cursor.execute("""
                    INSERT INTO user_results (id, result) VALUES (%s, 1)
                """, (id,))
                conn.commit()
            else:
                cursor.execute("""
                    INSERT INTO results (id, result) VALUES (%s, 0)
                """, (id,))
                conn.commit()

            # Delete the row from the arrays table
            cursor.execute("""
                DELETE FROM arrays WHERE id = %s
            """, (id,))
            conn.commit()

        # Wait for a bit before checking again
        time.sleep(0.001)

    cursor.close()
    conn.close()

poll_for_changes()
